In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!ls

adc.json  sample_data


In [ ]:
download = drive.CreateFile({'id': '15OCV9UbTR7k8ks-TDcC93r3bjhrG3xvo'})
download.GetContentFile('my_model.h5')

In [ ]:
#for download the dataset
# Please fill the ... with the id of Dataset_JPV.zip in the google drive

download = drive.CreateFile({'id': '15OCV9UbTR7k8ks-TDcC93r3bjhrG3xvo'})
download.GetContentFile('Dataset_JPV.zip')
download.GetContentFile('my_model.zip')

#!unzip rotated_3100.zip there is not file with that named

In [ ]:
# unzip your file
!unzip -o Dataset_JPV.zip -d Dataset_JPV > /dev/null
!unzip -o my_model.zip -d my_model > /dev/null


In [ ]:
# load data func
import glob
from keras.preprocessing import image as kImage

def getData(dataset_dir):
    X_list= sorted(glob.glob(os.path.join(dataset_dir, 'x','*.jpg')))
    Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
    
    X= []
    Y= []
    for i in range(len(X_list)):
        # Load input image
        x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        X.append(x)
        
        # Load ground-truth label and encode it to label 0 and 1
        y = kImage.load_img(Y_list[i], color_mode = "grayscale")#grayscale = True)
        y = kImage.img_to_array(y)
        y /= 255.0
        y = np.floor(y)# ?
        Y.append(y)
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    # Shuffle the training data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    return X, Y

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('my_mdel.h5', verbose=1, save_best_only=True)
#results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

In [ ]:
checkpointer

In [ ]:
model = load_model('my_mdel.h5')

OSError: ignored

In [ ]:
dataset_path = os.path.join('Dataset_JPV', 'test')
X, Y = getData(dataset_path)
# predict
pred = model.predict(X, verbose=1, batch_size=1)
print("Average Test Accuracy for Amir Dataset:" ,tf.Session().run(K.mean(K.equal(Y, K.round(pred)))))

NameError: ignored

In [ ]:
def get_model(num_classes, img_size=320, compile=True):
    print("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(240,img_size,3), name="input_1")
    layers = tf.keras.layers.Conv2D(64,(1,1), activation="relu")(inputs)
    layers = tf.keras.layers.Conv2D(64,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.MaxPool2D((2,2))(layers)
    layers = tf.keras.layers.Conv2D(128,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.Conv2D(128,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.MaxPool2D((2,2))(layers)
    layers = tf.keras.layers.Conv2D(256,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.Conv2D(256,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.MaxPool2D((2,2))(layers)
    layers = tf.keras.layers.Conv2D(512,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.Conv2D(512,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.MaxPool2D((2,2))(layers)
    layers = tf.keras.layers.Conv2D(512,(1,1), activation="relu")(layers)
    layers = tf.keras.layers.Conv2DTranspose(256,(30,40), activation="relu")(layers)
    layers = tf.keras.layers.Conv2DTranspose(128,(60,80), activation="relu")(layers)
    layers = tf.keras.layers.Conv2DTranspose(64,(120,160), activation="relu")(layers)
    layers = tf.keras.layers.Conv2DTranspose(32,(240,320), activation="relu")(layers)
    outputs = tf.keras.layers.Conv2DTranspose(1, (1, 1), activation='sigmoid')(layers)
    model = tf.keras.Model(inputs = inputs, outputs=outputs)
    if compile:
        model.compile(optimizer='adam',
                      loss='Binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [ ]:
def train(model, batch_size, epochs, model_name=""):
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/"+model_name+"_"+"{}".format(time()))
    model.reset_states()
    model.fit(x_train, y_train, epochs=epochs, callbacks=[tensorboard],
              batch_size=batch_size,
              validation_data=(x_test, y_test))
    metrics = model.evaluate(x_test, y_test)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [ ]:
num_classes = 33
model = get_model(num_classes)
model.summary()

using 33 classes
Model: "functional_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 320, 3)]     0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 240, 320, 64)      256       
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 240, 320, 64)      4160      
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 120, 160, 64)      0         
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 120, 160, 128)     8320      
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 120, 160, 128)     16512     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 

In [ ]:
from numpy import loadtxt
from keras.models import load_model

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('my_model.h5')

OSError: ignored

In [ ]:
import os
import tensorflow as tf
import numpy as np

dataset_path = os.path.join('Dataset_JPV', 'test')
X, Y = getData(dataset_path)
# predict
pred = model.predict(X, verbose=1, batch_size=1)
print("Average Test Accuracy for Amir Dataset:" ,tf.Session().run(K.mean(K.equal(Y, K.round(pred)))))

NameError: ignored